# Custom Tools Demo
<img src="https://eu-images.contentstack.com/v3/assets/blt6b0f74e5591baa03/blt5d5323f706ef288f/637c0195c162df49beaae102/Untitled_design_(77).png?width=1280&auto=webp&quality=95&format=jpg&disable=upscale" width=500 />

In [0]:
%pip install --upgrade --quiet langchain-core langchain-databricks langchain databricks-vectorsearch langchain-community youtube_search wikipedia feedparser
#probably don't need the youtbue or wikipedia libaries, but hey whatever. Might still use it.
%pip install transformers sentence_transformers

dbutils.library.restartPython()

## Simulating a few API Calls
We're going to build some tools around some API calls. For that we'll need to find a few good analogs for things like time series data
<br/>
Simulate time series retrieval<br/>
	•	REST API for weather forecasts and historical data.<br/>
	•	Simulates querying sensor data or time series like in Cognite.<br/>
GET https://api.open-meteo.com/v1/forecast?latitude=51.0478&longitude=114.0593&hourly=temperature_2m<br/>
<br/>
Simulate asset metadata catalog<br/>
	•	Can simulate fetching metadata about real-world locations (assets).<br/>
	•	Think of this as querying a catalog of objects.<br/>
GET https://nominatim.openstreetmap.org/search?q=Statue+of+Liberty&format=json<br/>
<br/>
Simulate a document search<br/>
	•	Useful for document-based retrieval like CDF events or files.<br/>
	•	Great for agent retrieval demos with RAG.<br/>
GET http://export.arxiv.org/api/query?search_query=all:ai&start=0&max_results=2<br/>

In [0]:
#Let's create some sample functions to test our endpoints. We can call these just to get a sense of how they behave and what kind of input parameters we'll need to inject.

import requests

def fetch_weather(latitude: float, longitude: float):
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m",
    }
    response = requests.get(url, params=params)
    return response.json()

def search_assets(query: str):
    url = "https://nominatim.openstreetmap.org/search"
    params = {"q": query, "format": "json"}
    response = requests.get(url, params=params)
    return response.json()

def search_arxiv(keywords: str, max_results: int = 3):
    url = "http://export.arxiv.org/api/query"
    params = {
        "search_query": f"all:{keywords}",
        "start": 0,
        "max_results": max_results
    }
    response = requests.get(url, params=params)
    return response.text  # XML format, you can parse with `feedparser`

In [0]:
#Let's do a quick preview of our data. The easiest way to do this is to throw it into a dataframe. We're just looking for a successful call and loose schema.
# We can do the same for any of our API calls

import pandas as pd

#Let's get a sample from Calgary
pd.DataFrame(fetch_weather(51.0478, 114.0593))

## (Option 1): Using LangChain's tools library to create tool stubs
This is a pretty easy approach, using a declarative approach and LangChain's tool library. All you need to do is register a function with a name and detailed description that will help the agent understand what each tool does.

This approach comes with a tradeoff; we lose flexibility with how our tools behave with the APIs. Since we'll want to tailor this behavior when we POST the payload, and retrieve only certain attributes this method is really only suitable for prototyping and rapid development.

In [0]:
#LangChain provides a quick and easy way to log functions as tools. This is an 'easy button' but we lose a degree of control in terms of behaviour management. Nonetheless, it's a pretty easy way to get going if you just want to do something simple.

#Although we probably won't be using this set of tools, it's good to know that it's an option for rapid development or prototyping.

from langchain.tools import Tool

tool_box = [
    Tool.from_function(fetch_weather, name="FetchWeather", description="Fetch hourly weather data."),
    Tool.from_function(search_assets, name="SearchAssets", description="Search for asset-like locations."),
    Tool.from_function(search_arxiv, name="SearchArxiv", description="Search research documents."),
]

## (Option 2): Creating our own tools using LangChain's Abstract BaseTool() class
There are some benefits to building our own tools based on abstract declarations from LangChain. Custom tools have added support for extended behaviours that simply aren't available when we register a tool using the from_function() .... function. This allows us to tailor our tools better to our needs, and we can build more complex functions that make external calls and have their own dependencies.

In [0]:
#Note: normally we wouldn't worry about re-declaring our library imports, but in this case we are so the cells can be copied and pasted elsewhere to other notebooks.

from langchain.tools import BaseTool
from typing import Optional, Type
from pydantic import BaseModel, Field
import requests

#Input Schema for the weather class. We'll be using this to compose the tool
class WeatherInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location")
    longitude: float = Field(..., description="Longitude of the location")

#Create an implementation inheriting the BaseTool abstract class from LangChain. _run() and _arun() are both required implementations. The name and description attributes are also required.
class FetchWeatherTool(BaseTool):
    name: str = "fetch_weather"
    description: str = "Fetch hourly weather temperature data for a given latitude and longitude. When asked about weather, assume that the user is always referring to temperature only. temperature_2m refers to the temperature in degrees celsius."
    args_schema: Type[BaseModel] = WeatherInput #This is the input schema we defined above.

    #The _run() function is always called when invoked. It's essentially doing the same thing as a class constructor, but since we're invoking the class from a toolchain in lieu of instancing the class as an object, this is run instead. This behaviour is what we're inheriting from the BaseTool() class.
    def _run(self, latitude: float, longitude: float):
        url = "https://api.open-meteo.com/v1/forecast"
        params = {
            "latitude": latitude,
            "longitude": longitude,
            "hourly": "temperature_2m",
        }
        response = requests.get(url, params=params)
        if response.status_code == 200:
            return response.json()["hourly"]["temperature_2m"][:5]  # Preview
        return f"Failed to fetch data: {response.status_code}"

    def _arun(self, *args, **kwargs):
        raise NotImplementedError("Async not supported.")

In [0]:
from langchain.tools import BaseTool
from typing import Optional, Type
from pydantic import BaseModel, Field
import requests
import feedparser


class ArxivSearchInput(BaseModel):
    query: str = Field(..., description="Search keywords for the arXiv research papers")
    max_results: Optional[int] = Field(5, description="Max number of papers to return")

class SearchArxivTool(BaseTool):
    name: str = "search_arxiv"
    description: str = "Search for academic papers on arXiv related to a given keyword."
    args_schema: Type[BaseModel] = ArxivSearchInput

    def _run(self, query: str, max_results: int = 3):
        url = "http://export.arxiv.org/api/query"
        params = {
            "search_query": f"all:{query}",
            "start": 0,
            "max_results": max_results,
        }
        response = requests.get(url, params=params)
        feed = feedparser.parse(response.text)
        results = []
        for entry in feed.entries[:max_results]:
            results.append(f"{entry.title} — {entry.link}")
        return "\n".join(results) if results else "No papers found."

    def _arun(self, *args, **kwargs):
        raise NotImplementedError("Async not supported.")


In [0]:
from langchain.tools import BaseTool
from typing import Optional, Type
from pydantic import BaseModel, Field
import requests

class AssetSearchInput(BaseModel):
    asset_query: str = Field(..., description="The name of the location or asset to search for")

class SearchAssetsTool(BaseTool):
    name: str = "search_assets"
    description: str = "Search for geographic or structural asset metadata using OpenStreetMap. "
    args_schema: Type[BaseModel] = AssetSearchInput

    def _run(self, asset_query: str):
        url = "https://nominatim.openstreetmap.org/search"
        params = {"q": asset_query, "format": "json"}
        headers = {"User-Agent": "LangChainAgent/1.0 (andrij.demianczuk@databricks.com)"}

        response = requests.get(url, params=params, headers=headers)
        if response.status_code != 200:
            return f"Search API returned {response.status_code}. Cannot continue using search_assets."
        
        data = response.json()
        if data:
            result = data[0]
            return f"{result['display_name']} (lat: {result['lat']}, lon: {result['lon']})"
        else:
            return f"No results found for '{asset_query}'"

    def _arun(self, *args, **kwargs):
        raise NotImplementedError("Async not supported.")

## Build our agent that will use the tools
Now that we have our toolboxes.
Once we have our tools registered and added to our toolbox (aka `tools[]`), we can now create an agent to make use of them. Think of tools at this point as a list of deterministic things that can be used to answer a question.

In [0]:
from langchain_databricks import ChatDatabricks

#This is the foundation LLM that we'll be using for the basis of our agents
LLM_ENDPOINT_NAME = "databricks-claude-3-7-sonnet"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME, extra_params={"temperature": 0.3})

In [0]:
print(llm.invoke('What are Calgarys coordinates?'))

## Agent Simulation 1: Weather & Research Articles

In [0]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType

import os
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

#Define the tools available to our agent
tools = [
    FetchWeatherTool(),
    SearchArxivTool(),
]

# Initialize conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=10,
    return_messages=True
)

aibot = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=10
)

In [0]:
new_prompt = aibot.agent.create_prompt(
    tools=tools
)

aibot.agent.llm_chain.prompt = new_prompt

## Agent Simulation 2: Custom Asset Lookups

In [0]:
#The second agent will be responsible for doing asset lookups. If we had a known set of our assets that our foundational model isn't aware of (e.g., OT asset locations in the field), we could use it to get our coordinates. When we create our composition of agents we'll declare descriptions for each agent.

from langchain.agents import initialize_agent
from langchain.agents import AgentType

import os
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

#We're going to use this to create a second agent for a multi-agent approach
asset_tools = [
    SearchAssetsTool()
]

# Initialize conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=10,
    return_messages=True
)

asset_bot = initialize_agent(
    tools=asset_tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=10
)

In [0]:
asset_prompt = asset_bot.agent.create_prompt(
    tools=asset_tools
)

asset_bot.agent.llm_chain.prompt = asset_prompt

## Testing our tools with both agents

In [0]:
aibot_output = aibot("Can you find me some article references for AI?")

In [0]:
print(aibot_output['output'])

In [0]:
aibot_output = aibot("Can you find me the address for the White House in Washington DC?")

In [0]:
print(aibot_output['output'])

In [0]:
aibot_output = aibot("What is the temperature at 51.0447° N, 114.0719° W?")

In [0]:
print(aibot_output['output'])

In [0]:
aibot_output = aibot("What are the lat and lon coordinates of West Edmonton Mall?")

In [0]:
print(aibot_output['output'])

In [0]:
aibot_output = aibot("What is the weather around West Edmonton Mall?")

In [0]:
print(aibot_output['output'])

In [0]:
asset_output = asset_bot("What are the coordinates of the Saddledome in Calgary AB?")

## Assembling our new agents into an compound AI system